In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time
from time import sleep
from random import randint
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
def parse_html(html):
    data, item = pd.DataFrame(), {}
    soup = BeautifulSoup(html,'lxml')
    for i, resto in enumerate(soup.find_all('div', class_='ui_column is-9')):
        
        date = str(resto.find('span',class_='ratingDate'))
        item['date'] = BeautifulSoup(date,"lxml").get_text() 
        
        review=str(resto.find('p',class_='partial_entry'))
        item['reviews']=BeautifulSoup(review, "lxml").get_text()
        
        rating_review=resto.select('.innerBubble .ui_bubble_rating')
        item['rating_review'] = rating_review[0]['class']
        
        data[i] = pd.Series(item)
    return data.T

In [3]:
reviews = pd.DataFrame()
driver = webdriver.Chrome()
driver.maximize_window()

url_first_page = 'https://www.tripadvisor.es/Restaurant_Review-g187497-d782944-Reviews-Cerveceria_Catalana-Barcelona_Catalonia.html'
driver.get(url_first_page)

x = randint(2,6)

sleep(x)
driver.find_elements_by_xpath("//*[@id='onetrust-accept-btn-handler']")[0].click()

sleep(x)
driver.find_elements_by_xpath("//*[@id='filters_detail_language_filterLang_en']")[0].click()

sleep(x)
driver.find_elements_by_xpath("//*[contains(@id, 'rn')]")[0].click()

# import time
# timeout = time.time() + 40*1 

while True:
#     if time.time() > timeout:
#         break
#     else:
        next_page_btn = driver.find_elements_by_xpath("//*[@id='taplc_location_reviews_list_sur_0']/div/div[10]/div/div/a[2]")
        if len(next_page_btn) < 1:
            print("No more pages left")
            break
        else:
            new_data = parse_html(driver.page_source)
            reviews = pd.concat([reviews, new_data], ignore_index=True)
            next_page_btn[0].click()
            x = randint(2,6)
            sleep(x)
            
        

driver.close()
reviews

No more pages left


,date,reviews,rating_review
0,Opinión escrita hace una semana,,"[ui_bubble_rating, bubble_50]"
1,Opinión escrita hace 2 semanas,,"[ui_bubble_rating, bubble_40]"
2,Opinión escrita hace 3 semanas,,"[ui_bubble_rating, bubble_20]"
3,Opinión escrita hace 3 semanas,,"[ui_bubble_rating, bubble_50]"
4,Opinión escrita hace 4 semanas,,"[ui_bubble_rating, bubble_50]"
...,...,...,...
5435,Opinión escrita 6 de enero de 2008,"Next to the shops, but very different from the...","[ui_bubble_rating, bubble_40]"
5436,Opinión escrita 28 de diciembre de 2007,Good tapas and always busy. Local place,"[ui_bubble_rating, bubble_30]"
5437,Opinión escrita 24 de diciembre de 2007,"everytime that I´m visiting Barcelona, got to go!","[ui_bubble_rating, bubble_50]"
5438,Opinión escrita 21 de diciembre de 2007,Very well known for the business men-women. It...,"[ui_bubble_rating, bubble_40]"


In [4]:
reviews2 = pd.DataFrame()
driver = webdriver.Chrome()
driver.maximize_window()

def parse_html2(html):
    data, item = pd.DataFrame(), {}
    soup = BeautifulSoup(html,'lxml') 
    for i, resto in enumerate(soup.find_all('div', class_='ui_column is-9')):

        review=str(resto.find('p',class_='partial_entry'))
        item['reviews2']=BeautifulSoup(review, "lxml").get_text()

        data[i] = pd.Series(item)
    return data.T

url_first_page = 'https://www.tripadvisor.es/ShowUserReviews-g187497-d782944-r830535155-Cerveceria_Catalana-Barcelona_Catalonia.html'
driver.get(url_first_page)

x = randint(2,6)

sleep(x)
driver.find_elements_by_xpath("//*[@id='onetrust-accept-btn-handler']")[0].click()

new_data = parse_html2(driver.page_source)
reviews2 = pd.concat([reviews2, new_data], ignore_index=True)
driver.close()

reviews2


,reviews2
0,This restaurant was recommended to us by our h...
1,Had to wait for a table but glad we did. The f...
2,We went here on the last night of a 4 night tr...
3,Our hotel recommended this restaurant. It was ...
4,Fantastic meal at Cervecería Catalana. Food wa...


In [5]:
reviews.to_csv('reviews.csv',encoding='utf-8-sig')

In [6]:
reviews=pd.read_csv('reviews.csv')

In [7]:
allreviews = pd.DataFrame(reviews['reviews'].fillna(reviews2['reviews2']))


In [8]:
allreviews.rename(columns={'reviews': 'name'}, inplace=True)
allreviews

,name
0,This restaurant was recommended to us by our h...
1,Had to wait for a table but glad we did. The f...
2,We went here on the last night of a 4 night tr...
3,Our hotel recommended this restaurant. It was ...
4,Fantastic meal at Cervecería Catalana. Food wa...
...,...
5435,"Next to the shops, but very different from the..."
5436,Good tapas and always busy. Local place
5437,"everytime that I´m visiting Barcelona, got to go!"
5438,Very well known for the business men-women. It...


In [9]:
reviews=pd.concat([reviews, allreviews['name']], axis=1)
reviews

,Unnamed: 0,date,reviews,rating_review,name
0,0,Opinión escrita hace una semana,NaN,"['ui_bubble_rating', 'bubble_50']",This restaurant was recommended to us by our h...
1,1,Opinión escrita hace 2 semanas,NaN,"['ui_bubble_rating', 'bubble_40']",Had to wait for a table but glad we did. The f...
2,2,Opinión escrita hace 3 semanas,NaN,"['ui_bubble_rating', 'bubble_20']",We went here on the last night of a 4 night tr...
3,3,Opinión escrita hace 3 semanas,NaN,"['ui_bubble_rating', 'bubble_50']",Our hotel recommended this restaurant. It was ...
4,4,Opinión escrita hace 4 semanas,NaN,"['ui_bubble_rating', 'bubble_50']",Fantastic meal at Cervecería Catalana. Food wa...
...,...,...,...,...,...
5435,5435,Opinión escrita 6 de enero de 2008,"Next to the shops, but very different from the...","['ui_bubble_rating', 'bubble_40']","Next to the shops, but very different from the..."
5436,5436,Opinión escrita 28 de diciembre de 2007,Good tapas and always busy. Local place,"['ui_bubble_rating', 'bubble_30']",Good tapas and always busy. Local place
5437,5437,Opinión escrita 24 de diciembre de 2007,"everytime that I´m visiting Barcelona, got to go!","['ui_bubble_rating', 'bubble_50']","everytime that I´m visiting Barcelona, got to go!"
5438,5438,Opinión escrita 21 de diciembre de 2007,Very well known for the business men-women. It...,"['ui_bubble_rating', 'bubble_40']",Very well known for the business men-women. It...


In [10]:
reviews.drop(['Unnamed: 0','reviews'], axis = 1, inplace = True)
reviews.rename(columns={'name': 'reviews'}, inplace=True)
reviews

,date,rating_review,reviews
0,Opinión escrita hace una semana,"['ui_bubble_rating', 'bubble_50']",This restaurant was recommended to us by our h...
1,Opinión escrita hace 2 semanas,"['ui_bubble_rating', 'bubble_40']",Had to wait for a table but glad we did. The f...
2,Opinión escrita hace 3 semanas,"['ui_bubble_rating', 'bubble_20']",We went here on the last night of a 4 night tr...
3,Opinión escrita hace 3 semanas,"['ui_bubble_rating', 'bubble_50']",Our hotel recommended this restaurant. It was ...
4,Opinión escrita hace 4 semanas,"['ui_bubble_rating', 'bubble_50']",Fantastic meal at Cervecería Catalana. Food wa...
...,...,...,...
5435,Opinión escrita 6 de enero de 2008,"['ui_bubble_rating', 'bubble_40']","Next to the shops, but very different from the..."
5436,Opinión escrita 28 de diciembre de 2007,"['ui_bubble_rating', 'bubble_30']",Good tapas and always busy. Local place
5437,Opinión escrita 24 de diciembre de 2007,"['ui_bubble_rating', 'bubble_50']","everytime that I´m visiting Barcelona, got to go!"
5438,Opinión escrita 21 de diciembre de 2007,"['ui_bubble_rating', 'bubble_40']",Very well known for the business men-women. It...


In [11]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5440 entries, 0 to 5439
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           5440 non-null   object
 1   rating_review  5440 non-null   object
 2   reviews        5440 non-null   object
dtypes: object(3)
memory usage: 127.6+ KB


In [12]:
# pd.set_option("display.max_colwidth", None)
# pd.set_option('display.max_rows', None)
reviews

,date,rating_review,reviews
0,Opinión escrita hace una semana,"['ui_bubble_rating', 'bubble_50']",This restaurant was recommended to us by our h...
1,Opinión escrita hace 2 semanas,"['ui_bubble_rating', 'bubble_40']",Had to wait for a table but glad we did. The f...
2,Opinión escrita hace 3 semanas,"['ui_bubble_rating', 'bubble_20']",We went here on the last night of a 4 night tr...
3,Opinión escrita hace 3 semanas,"['ui_bubble_rating', 'bubble_50']",Our hotel recommended this restaurant. It was ...
4,Opinión escrita hace 4 semanas,"['ui_bubble_rating', 'bubble_50']",Fantastic meal at Cervecería Catalana. Food wa...
...,...,...,...
5435,Opinión escrita 6 de enero de 2008,"['ui_bubble_rating', 'bubble_40']","Next to the shops, but very different from the..."
5436,Opinión escrita 28 de diciembre de 2007,"['ui_bubble_rating', 'bubble_30']",Good tapas and always busy. Local place
5437,Opinión escrita 24 de diciembre de 2007,"['ui_bubble_rating', 'bubble_50']","everytime that I´m visiting Barcelona, got to go!"
5438,Opinión escrita 21 de diciembre de 2007,"['ui_bubble_rating', 'bubble_40']",Very well known for the business men-women. It...


In [13]:
type(reviews['rating_review'])

pandas.core.series.Series

In [14]:
reviews['rating_review']=reviews['rating_review'].apply(str)
reviews['rating_review']

0       ['ui_bubble_rating', 'bubble_50']
1       ['ui_bubble_rating', 'bubble_40']
2       ['ui_bubble_rating', 'bubble_20']
3       ['ui_bubble_rating', 'bubble_50']
4       ['ui_bubble_rating', 'bubble_50']
                      ...                
5435    ['ui_bubble_rating', 'bubble_40']
5436    ['ui_bubble_rating', 'bubble_30']
5437    ['ui_bubble_rating', 'bubble_50']
5438    ['ui_bubble_rating', 'bubble_40']
5439    ['ui_bubble_rating', 'bubble_50']
Name: rating_review, Length: 5440, dtype: object

In [15]:
reviews['rating_review']=reviews['rating_review'].str.extract(pat = '([0-9].)')
reviews

,date,rating_review,reviews
0,Opinión escrita hace una semana,50,This restaurant was recommended to us by our h...
1,Opinión escrita hace 2 semanas,40,Had to wait for a table but glad we did. The f...
2,Opinión escrita hace 3 semanas,20,We went here on the last night of a 4 night tr...
3,Opinión escrita hace 3 semanas,50,Our hotel recommended this restaurant. It was ...
4,Opinión escrita hace 4 semanas,50,Fantastic meal at Cervecería Catalana. Food wa...
...,...,...,...
5435,Opinión escrita 6 de enero de 2008,40,"Next to the shops, but very different from the..."
5436,Opinión escrita 28 de diciembre de 2007,30,Good tapas and always busy. Local place
5437,Opinión escrita 24 de diciembre de 2007,50,"everytime that I´m visiting Barcelona, got to go!"
5438,Opinión escrita 21 de diciembre de 2007,40,Very well known for the business men-women. It...


In [16]:
reviews['reviews'].apply(str)

0       This restaurant was recommended to us by our h...
1       Had to wait for a table but glad we did. The f...
2       We went here on the last night of a 4 night tr...
3       Our hotel recommended this restaurant. It was ...
4       Fantastic meal at Cervecería Catalana. Food wa...
                              ...                        
5435    Next to the shops, but very different from the...
5436              Good tapas and always busy. Local place
5437    everytime that I´m visiting Barcelona, got to go!
5438    Very well known for the business men-women. It...
5439                        one of my favourite tapas bar
Name: reviews, Length: 5440, dtype: object

In [17]:
pd.set_option("display.max_colwidth", None)

In [18]:
reviews['reviews']

0                                                                                                                                                                                                                                                                                                                                                                                                      This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and "fresh today" dishes. Well priced and worth a visit. 
1                                                                                                                                                                                                 Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef 

In [19]:
reviews

,date,rating_review,reviews
0,Opinión escrita hace una semana,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit."
1,Opinión escrita hace 2 semanas,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try."
2,Opinión escrita hace 3 semanas,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!"
3,Opinión escrita hace 3 semanas,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!"
4,Opinión escrita hace 4 semanas,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend."
...,...,...,...
5435,Opinión escrita 6 de enero de 2008,40,"Next to the shops, but very different from the turist traps you find everywhere. Service is good and fast, the place is cosy and food - yummy! The best dish (tapas) was the one that had fish and cheese in a pepper. So good I need to get there again!"
5436,Opinión escrita 28 de diciembre de 2007,30,Good tapas and always busy. Local place
5437,Opinión escrita 24 de diciembre de 2007,50,"everytime that I´m visiting Barcelona, got to go!"
5438,Opinión escrita 21 de diciembre de 2007,40,Very well known for the business men-women. It's always plenty of 25-30 years old people after day-working!


In [20]:
reviews = reviews.replace(r'Opinión escrita ', '', regex = True)
reviews = reviews.replace(r'hace ', '', regex = True)
reviews = reviews.replace(r' de enero de ', '-01-', regex = True)
reviews = reviews.replace(r' de febrero de ', '-02-', regex = True)
reviews = reviews.replace(r' de marzo de ', '-03-', regex = True)
reviews = reviews.replace(r' de abril de ', '-04-', regex = True)
reviews = reviews.replace(r' de mayo de ', '-05-', regex = True)
reviews = reviews.replace(r' de junio de ', '-06-', regex = True)
reviews = reviews.replace(r' de julio de ', '-07-', regex = True)
reviews = reviews.replace(r' de agosto de ', '-08-', regex = True)
reviews = reviews.replace(r' de septiembre de ', '-09-', regex = True)
reviews = reviews.replace(r' de octubre de ', '-10-', regex = True)
reviews = reviews.replace(r' de noviembre de ', '-11-', regex = True)
reviews = reviews.replace(r' de diciembre de ', '-12-', regex = True)
reviews = reviews.replace(r'un día', '1 day', regex = True)
reviews = reviews.replace(r'días', 'day', regex = True)
reviews = reviews.replace(r'una semana', '7 day', regex = True)
reviews = reviews.replace(r'semanas', 'week', regex = True)
reviews

,date,rating_review,reviews
0,7 day,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit."
1,2 week,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try."
2,3 week,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!"
3,3 week,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!"
4,4 week,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend."
...,...,...,...
5435,6-01-2008,40,"Next to the shops, but very different from the turist traps you find everywhere. Service is good and fast, the place is cosy and food - yummy! The best dish (tapas) was the one that had fish and cheese in a pepper. So good I need to get there again!"
5436,28-12-2007,30,Good tapas and always busy. Local place
5437,24-12-2007,50,"everytime that I´m visiting Barcelona, got to go!"
5438,21-12-2007,40,Very well known for the business men-women. It's always plenty of 25-30 years old people after day-working!


In [21]:
day=reviews[reviews['date'].str.contains(r'day')]
day

,date,rating_review,reviews
0,7 day,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit."


In [22]:
week=reviews[reviews['date'].str.contains(r'week')]
week

,date,rating_review,reviews
1,2 week,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try."
2,3 week,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!"
3,3 week,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!"
4,4 week,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend."


In [23]:
day['date'] = day['date'].replace(r' day', '', regex = True)
day

,date,rating_review,reviews
0,7,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit."


In [24]:
week['date']=week['date'].replace(r' week','',regex = True)
week['date']=week['date'].astype(int)
week['date']=week['date']*7
week

,date,rating_review,reviews
1,14,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try."
2,21,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!"
3,21,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!"
4,28,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend."


In [25]:
dayweek=pd.concat([day, week], ignore_index=True)
dayweek

,date,rating_review,reviews
0,7,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit."
1,14,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try."
2,21,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!"
3,21,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!"
4,28,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend."


In [26]:
dayweek['date'].dtype

dtype('O')

In [27]:
dw=dayweek['date'].astype(int)

In [28]:
dw

0     7
1    14
2    21
3    21
4    28
Name: date, dtype: int32

In [29]:
import datetime

today = datetime.date.today()
listdw=[]

for i in dw:
    original_date = today - datetime.timedelta(days=i)
    formatted_date = original_date.strftime("%d-%m-%Y")
    listdw.append(formatted_date)
    
dayweek['date'] = listdw
dayweek

,date,rating_review,reviews
0,10-03-2022,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit."
1,03-03-2022,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try."
2,24-02-2022,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!"
3,24-02-2022,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!"
4,17-02-2022,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend."


In [30]:
rest_dayweek=reviews[~reviews['date'].str.contains(r'day|week')]
rest_dayweek

,date,rating_review,reviews
5,7-02-2022,50,"I’d seen the reviews online and had a friend recommendation to try here and I’m glad I did! The garlic prawns with tomato bread were the best! I also had some ham/cheese tapas, a beer and finished with some churos. It was 25 euros so well worth it. Would definitely go back! The inside is more bright and modern than I was expecting but it had a nice atmosphere and great food."
6,29-01-2022,50,"That’s an excellent place. High quality small dishes, with a very wide offer. It is always full, you need to queue. Go with plenty of time. The atmosphere is great, service is fast and prices are reasonable. You can have a taste of the local cuisine with several small dishes. You will not go there only once."
7,25-01-2022,50,"We found this restaurant by chance and we loved it! Very good place with quality food and very good treatment by the waiters. For dessert I ordered the crema catalana and it is the best I've ever tasted, it was only 2.95 euros, amazing!"
8,20-01-2022,50,The Cerveceria Catalana was simply 5 Stars in any category. The Tapas were outstanding. The service was outstanding. And the prices were excellent. We were in Barcelona 2 nights and ate there both nights. We really enjoyed our experience.
9,29-12-2021,40,"Great ' classic' tapas, almost all dished we ordered were amazing. Food comes very quickly, best to not order too much at the same time. Staff is friendly, however can appear a bit rushed as this place seems to have a waiting line outside most of the time. But really a must-try!"
...,...,...,...
5435,6-01-2008,40,"Next to the shops, but very different from the turist traps you find everywhere. Service is good and fast, the place is cosy and food - yummy! The best dish (tapas) was the one that had fish and cheese in a pepper. So good I need to get there again!"
5436,28-12-2007,30,Good tapas and always busy. Local place
5437,24-12-2007,50,"everytime that I´m visiting Barcelona, got to go!"
5438,21-12-2007,40,Very well known for the business men-women. It's always plenty of 25-30 years old people after day-working!


In [31]:
rest_dayweek['date'].dtypes

dtype('O')

In [32]:
dayweek['date'].dtypes

dtype('O')

In [33]:
review_final=pd.concat([dayweek, rest_dayweek], axis=0)
review_final

,date,rating_review,reviews
0,10-03-2022,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit."
1,03-03-2022,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try."
2,24-02-2022,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!"
3,24-02-2022,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!"
4,17-02-2022,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend."
...,...,...,...
5435,6-01-2008,40,"Next to the shops, but very different from the turist traps you find everywhere. Service is good and fast, the place is cosy and food - yummy! The best dish (tapas) was the one that had fish and cheese in a pepper. So good I need to get there again!"
5436,28-12-2007,30,Good tapas and always busy. Local place
5437,24-12-2007,50,"everytime that I´m visiting Barcelona, got to go!"
5438,21-12-2007,40,Very well known for the business men-women. It's always plenty of 25-30 years old people after day-working!


In [34]:
#SENTIMENT ANALYSIS

In [35]:
from textblob import TextBlob

df=review_final
df['polarity_score'] = df['reviews'].apply(lambda x:TextBlob(str(x)).sentiment.polarity)
df['subjectivity_score'] = df['reviews'].apply(lambda x:TextBlob(x).sentiment.subjectivity)
df

,date,rating_review,reviews,polarity_score,subjectivity_score
0,10-03-2022,50,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit.",0.362338,0.428571
1,03-03-2022,40,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try.",0.468254,0.580952
2,24-02-2022,20,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!",-0.097361,0.443000
3,24-02-2022,50,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!",0.410625,0.685000
4,17-02-2022,50,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend.",0.515714,0.720000
...,...,...,...,...,...
5435,6-01-2008,40,"Next to the shops, but very different from the turist traps you find everywhere. Service is good and fast, the place is cosy and food - yummy! The best dish (tapas) was the one that had fish and cheese in a pepper. So good I need to get there again!",0.470833,0.480000
5436,28-12-2007,30,Good tapas and always busy. Local place,0.266667,0.300000
5437,24-12-2007,50,"everytime that I´m visiting Barcelona, got to go!",0.000000,0.000000
5438,21-12-2007,40,Very well known for the business men-women. It's always plenty of 25-30 years old people after day-working!,0.162500,0.250000


In [36]:
df.describe()

,polarity_score,subjectivity_score
count,5440.000000,5440.000000
mean,0.331133,0.544070
std,0.191589,0.135143
min,-0.750000,0.000000
25%,0.214905,0.462528
50%,0.322417,0.538896
75%,0.442937,0.621579
max,1.000000,1.000000


In [37]:
max_score=df.iloc[df['polarity_score'].idxmax()]
pd.DataFrame(max_score)

,164
date,23-10-2019
rating_review,50
reviews,Best Tapas in Barcelona!!!! Everything what we tried during visit to Barcelona was excellent. We visited this restaurant three times in 5 days!
polarity_score,1.0
subjectivity_score,0.65


In [38]:
min_score=df.iloc[df['polarity_score'].idxmin()]
pd.DataFrame(min_score)

,2826
date,23-09-2015
rating_review,20
reviews,"May be my choice was wrong but the food was not tasty at all. We have ordered an beef steak, mushrooms and assortment of sea food, that was just terrible."
polarity_score,-0.75
subjectivity_score,0.95


In [39]:
df[df['polarity_score'] < 0].count()

date                  181
rating_review         181
reviews               181
polarity_score        181
subjectivity_score    181
dtype: int64

In [40]:
df[df['polarity_score'] > 0].count()

date                  5231
rating_review         5231
reviews               5231
polarity_score        5231
subjectivity_score    5231
dtype: int64

In [41]:
df[df['polarity_score'] == 0].count()

date                  28
rating_review         28
reviews               28
polarity_score        28
subjectivity_score    28
dtype: int64

In [42]:
df['polarity_score'].mean() # from -1 to 1 --- 0.33 is positive more than the half (half is 0) -- 66,5 positive

0.3311333148299321

In [45]:
df['rating_review']=df['rating_review'].astype(int)

In [46]:
df['rating_review']=df['rating_review']/10
df

,date,rating_review,reviews,polarity_score,subjectivity_score
0,10-03-2022,5.0,"This restaurant was recommended to us by our hotel. The atmosphere was lively and busy and the food was excellent, a good mix of standard tapas and ""fresh today"" dishes. Well priced and worth a visit.",0.362338,0.428571
1,03-03-2022,4.0,"Had to wait for a table but glad we did. The food was really good, mixture of locals and tourists but nice atmosphere. We has a good selection of tapas including calamari’s, mixed fried fish, iberico ham and beef with foie gras along with tomato bread. Bill for 2 with a beer 4 large glasses of white wine and water plus all the tapas was 58 euros. Good value for great quality food. Give it a try.",0.468254,0.580952
2,24-02-2022,2.0,"We went here on the last night of a 4 night trip to Barcelona and were very disappointed. We arrived early (so we didn't have to queue!)- sat in the back, annoyed the waiter by asking for water (tiniest bottles ever!) and ordered all of our faves- shrimp in garlic (good), tomato bread (average), tortilla (average), ham (good)....overall, I would definitely not go back. Felt super touristy and food was so very average- almost like cafeteria food! We had much better tapas at other places on our trip. Very disappointed for our last meal there. Don't get what all the hype is about!",-0.097361,0.443000
3,24-02-2022,5.0,"Our hotel recommended this restaurant. It was fabulous! Great tapas at a very reasonable price. We were here twice and had different items each time. All were excellent! Nice atmosphere too. There’s usually a que, but well worth the wait!",0.410625,0.685000
4,17-02-2022,5.0,"Fantastic meal at Cervecería Catalana. Food was excellent, service good and really cool atmosphere. Great selection of tapas and reasonably priced. Highly recommend.",0.515714,0.720000
...,...,...,...,...,...
5435,6-01-2008,4.0,"Next to the shops, but very different from the turist traps you find everywhere. Service is good and fast, the place is cosy and food - yummy! The best dish (tapas) was the one that had fish and cheese in a pepper. So good I need to get there again!",0.470833,0.480000
5436,28-12-2007,3.0,Good tapas and always busy. Local place,0.266667,0.300000
5437,24-12-2007,5.0,"everytime that I´m visiting Barcelona, got to go!",0.000000,0.000000
5438,21-12-2007,4.0,Very well known for the business men-women. It's always plenty of 25-30 years old people after day-working!,0.162500,0.250000


In [89]:
df['rating_review'].value_counts()

5.0    3386
4.0    1417
3.0     372
2.0     133
1.0     132
Name: rating_review, dtype: int64

In [90]:
df['rating_review'].mean() # from 1 to 5 --- 4.43 is positive more than the half (half is 2.5) -- 88,6 positive

4.432352941176471

In [47]:
#WORK IN TABLEAU

In [48]:
df.to_csv('SentimentAnalysis.csv',encoding='utf-8-sig')

In [49]:
import nltk # underlying library NLP tools
from nltk.tokenize import word_tokenize

In [50]:
#nltk.download('all')

In [51]:
tokens=df['reviews'].apply(word_tokenize)
tokens

0                                                                                                                                                                                                                                                                                                                                [This, restaurant, was, recommended, to, us, by, our, hotel, ., The, atmosphere, was, lively, and, busy, and, the, food, was, excellent, ,, a, good, mix, of, standard, tapas, and, ``, fresh, today, '', dishes, ., Well, priced, and, worth, a, visit, .]
1                                                                         [Had, to, wait, for, a, table, but, glad, we, did, ., The, food, was, really, good, ,, mixture, of, locals, and, tourists, but, nice, atmosphere, ., We, has, a, good, selection, of, tapas, including, calamari, ’, s, ,, mixed, fried, fish, ,, iberico, ham, and, beef, with, foie, gras, along, with, tomato, bread, ., Bill, for, 2, with, a, beer, 4, large

In [52]:
type(tokens)

pandas.core.series.Series

In [53]:
result = []
[result.extend(i) for i in tokens] 
result

['This',
 'restaurant',
 'was',
 'recommended',
 'to',
 'us',
 'by',
 'our',
 'hotel',
 '.',
 'The',
 'atmosphere',
 'was',
 'lively',
 'and',
 'busy',
 'and',
 'the',
 'food',
 'was',
 'excellent',
 ',',
 'a',
 'good',
 'mix',
 'of',
 'standard',
 'tapas',
 'and',
 '``',
 'fresh',
 'today',
 "''",
 'dishes',
 '.',
 'Well',
 'priced',
 'and',
 'worth',
 'a',
 'visit',
 '.',
 'Had',
 'to',
 'wait',
 'for',
 'a',
 'table',
 'but',
 'glad',
 'we',
 'did',
 '.',
 'The',
 'food',
 'was',
 'really',
 'good',
 ',',
 'mixture',
 'of',
 'locals',
 'and',
 'tourists',
 'but',
 'nice',
 'atmosphere',
 '.',
 'We',
 'has',
 'a',
 'good',
 'selection',
 'of',
 'tapas',
 'including',
 'calamari',
 '’',
 's',
 ',',
 'mixed',
 'fried',
 'fish',
 ',',
 'iberico',
 'ham',
 'and',
 'beef',
 'with',
 'foie',
 'gras',
 'along',
 'with',
 'tomato',
 'bread',
 '.',
 'Bill',
 'for',
 '2',
 'with',
 'a',
 'beer',
 '4',
 'large',
 'glasses',
 'of',
 'white',
 'wine',
 'and',
 'water',
 'plus',
 'all',
 'the',
 '

In [54]:
tokens_lower=[token.lower() for token in result]

In [55]:
tokens_lower

['this',
 'restaurant',
 'was',
 'recommended',
 'to',
 'us',
 'by',
 'our',
 'hotel',
 '.',
 'the',
 'atmosphere',
 'was',
 'lively',
 'and',
 'busy',
 'and',
 'the',
 'food',
 'was',
 'excellent',
 ',',
 'a',
 'good',
 'mix',
 'of',
 'standard',
 'tapas',
 'and',
 '``',
 'fresh',
 'today',
 "''",
 'dishes',
 '.',
 'well',
 'priced',
 'and',
 'worth',
 'a',
 'visit',
 '.',
 'had',
 'to',
 'wait',
 'for',
 'a',
 'table',
 'but',
 'glad',
 'we',
 'did',
 '.',
 'the',
 'food',
 'was',
 'really',
 'good',
 ',',
 'mixture',
 'of',
 'locals',
 'and',
 'tourists',
 'but',
 'nice',
 'atmosphere',
 '.',
 'we',
 'has',
 'a',
 'good',
 'selection',
 'of',
 'tapas',
 'including',
 'calamari',
 '’',
 's',
 ',',
 'mixed',
 'fried',
 'fish',
 ',',
 'iberico',
 'ham',
 'and',
 'beef',
 'with',
 'foie',
 'gras',
 'along',
 'with',
 'tomato',
 'bread',
 '.',
 'bill',
 'for',
 '2',
 'with',
 'a',
 'beer',
 '4',
 'large',
 'glasses',
 'of',
 'white',
 'wine',
 'and',
 'water',
 'plus',
 'all',
 'the',
 '

In [56]:
len(tokens_lower)

460276

In [57]:
# remove punctuation
clean_tokens_lower=[word for word in tokens_lower if word.isalpha()]

In [58]:
clean_tokens_lower

['this',
 'restaurant',
 'was',
 'recommended',
 'to',
 'us',
 'by',
 'our',
 'hotel',
 'the',
 'atmosphere',
 'was',
 'lively',
 'and',
 'busy',
 'and',
 'the',
 'food',
 'was',
 'excellent',
 'a',
 'good',
 'mix',
 'of',
 'standard',
 'tapas',
 'and',
 'fresh',
 'today',
 'dishes',
 'well',
 'priced',
 'and',
 'worth',
 'a',
 'visit',
 'had',
 'to',
 'wait',
 'for',
 'a',
 'table',
 'but',
 'glad',
 'we',
 'did',
 'the',
 'food',
 'was',
 'really',
 'good',
 'mixture',
 'of',
 'locals',
 'and',
 'tourists',
 'but',
 'nice',
 'atmosphere',
 'we',
 'has',
 'a',
 'good',
 'selection',
 'of',
 'tapas',
 'including',
 'calamari',
 's',
 'mixed',
 'fried',
 'fish',
 'iberico',
 'ham',
 'and',
 'beef',
 'with',
 'foie',
 'gras',
 'along',
 'with',
 'tomato',
 'bread',
 'bill',
 'for',
 'with',
 'a',
 'beer',
 'large',
 'glasses',
 'of',
 'white',
 'wine',
 'and',
 'water',
 'plus',
 'all',
 'the',
 'tapas',
 'was',
 'euros',
 'good',
 'value',
 'for',
 'great',
 'quality',
 'food',
 'give',

In [59]:
# word frequency summary

In [60]:
len(tokens_lower)

460276

In [61]:
len(clean_tokens_lower)

396559

In [62]:
#remove stopwords
from nltk.corpus import stopwords

In [63]:
stop_words=stopwords.words('english')

In [64]:
clean_tokens_lower_without_stop=[word for word in clean_tokens_lower if not word in stop_words]

In [65]:
len(clean_tokens_lower_without_stop)

197763

In [66]:
#rename the variable 
new_words=clean_tokens_lower_without_stop

In [67]:
df_new_words=pd.DataFrame(new_words)
df_new_words.to_csv('words.csv',encoding='utf-8-sig')

In [68]:
#Most common words

In [69]:
freqdist2=nltk.FreqDist(new_words)

In [70]:
word_freq2=dict((word,freq) for word, freq in freqdist2.items() if not word.isdigit())

In [71]:
sorted_word_freq2=sorted(word_freq2.items(), key=lambda x:x[1], reverse=True)

In [72]:
sorted_word_freq2

[('tapas', 4714),
 ('food', 3555),
 ('place', 3015),
 ('good', 2957),
 ('wait', 2730),
 ('great', 2220),
 ('barcelona', 1989),
 ('restaurant', 1937),
 ('table', 1915),
 ('service', 1801),
 ('bar', 1500),
 ('time', 1428),
 ('go', 1374),
 ('get', 1346),
 ('worth', 1197),
 ('one', 1144),
 ('best', 1127),
 ('us', 1114),
 ('went', 1111),
 ('would', 1073),
 ('really', 964),
 ('delicious', 959),
 ('busy', 936),
 ('back', 909),
 ('staff', 885),
 ('excellent', 847),
 ('waiting', 846),
 ('people', 845),
 ('well', 841),
 ('try', 820),
 ('also', 783),
 ('menu', 783),
 ('around', 761),
 ('dishes', 758),
 ('nice', 750),
 ('recommend', 702),
 ('like', 692),
 ('recommended', 686),
 ('amazing', 680),
 ('minutes', 676),
 ('long', 668),
 ('ordered', 664),
 ('eat', 662),
 ('atmosphere', 656),
 ('first', 652),
 ('always', 633),
 ('order', 625),
 ('hour', 625),
 ('friendly', 621),
 ('wine', 619),
 ('lunch', 611),
 ('sangria', 607),
 ('got', 598),
 ('fresh', 589),
 ('dinner', 586),
 ('visit', 585),
 ('night'

In [86]:
top_words=pd.DataFrame(sorted_word_freq2,columns=['Top_Words', 'Frequency']) 

In [87]:
len(top_words)
# top_words=top_words.head(200)

9632

In [85]:
# top_words.to_csv('top_words.csv',encoding='utf-8-sig')

In [73]:
from nltk.text import Text

In [74]:
nltktext=Text(new_words)

In [75]:
type(new_words)

list

In [76]:
#collocations - words that appear next to eachother frequently 
nltktext.collocations()

cerveceria catalana; reasonably priced; patatas bravas; get table;
razor clams; worth wait; put name; highly recommend; foie gras; beef
tenderloin; come back; best tapas; value money; highly recommended;
take reservations; well worth; make sure; one best; would recommend;
wait table


In [77]:
# concordance - fragments containing key words - indexed - text around that word
nltktext.concordance("tapas")

Displaying 25 of 4714 matches:
usy food excellent good mix standard tapas fresh today dishes well priced worth
rists nice atmosphere good selection tapas including calamari mixed fried fish 
 large glasses white wine water plus tapas euros good value great quality food 
most like cafeteria food much better tapas places trip disappointed last meal g
ecommended restaurant fabulous great tapas reasonable price twice different ite
ally cool atmosphere great selection tapas reasonably priced highly recommend s
garlic prawns tomato bread best also tapas beer finished churos euros well wort
ceria catalana simply stars category tapas outstanding service outstanding pric
lly enjoyed experience great classic tapas almost dished ordered amazing food c
 service also good recommend enjoyed tapas restaurant everything good especiall
in walk still best recommended place tapas tripadvisor search reservations wait
each somewhere quit mesmerizing view tapas served outside tables left made even
ouple sit